In [43]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, select
from sqlalchemy.exc import SQLAlchemyError

# pandas formatting
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', 200)

In [2]:
dialect='mysql'
driver='mysqlconnector'
username='root'
password='root'
host='localhost'
port='3306'
database='dmapps_0'

engine = create_engine(f'{dialect}+{driver}://{username}:{password}@{host}:{port}/{database}')

In [3]:
def sql(query, engine):
    with engine.connect() as conn:
        return pd.read_sql(query, conn)

In [4]:
metadata = MetaData()
metadata.reflect(bind=engine)

In [13]:
results = []
matched_tables = set()

search_word = 'aquaculture'

with engine.connect() as connection:
    for table_name, table in metadata.tables.items():
        table_matched = False
        for column in table.columns:
            if str(column.type).startswith(("VARCHAR", "TEXT", "CHAR")):
                try:
                    query = select(table).where(column.ilike(f"%{search_word}%"))
                    result = connection.execute(query)
                    
                    if result.fetchone():
                        results.append((table_name, column.name))
                        table_matched = True
                except SQLAlchemyError as e:
                    print(f"Skipping {table_name}.{column.name}: {e}")
                    
        if table_matched:
            matched_tables.add(table_name)
    
    for table_name, table in metadata.tables.items():
        if table_name in matched_tables:
            for column in table.columns:
                if column.name.lower().endswith("id"):
                    results.append((table_name, column.name))


In [24]:
columns = dict()

for table, column in results:
    if columns.get(table, None):
        columns[table].add(column)
    else:
        columns[table] = {column}

table_list = sorted(list(matched_tables))

for table in table_list:
    col_list = sorted(list(columns[table]))
    lst1 = [x for x in col_list if x == 'id']
    lst2 = [x for x in col_list if x[-2:] == 'id' and x != 'id']
    lst3 = [x for x in col_list if x[-2:] != 'id']
    columns[table] = lst1 + lst2 + lst3
    

In [40]:
df_collection = dict()
terms = ['fisheries', 'aquatic ecosystems', 'marine mammal', 'aquaculture']
pattern = "|".join(terms)

for table in table_list:
    temp_columns = []
    for column in columns[table]:
        temp_columns += [column]
        
    query = f"SELECT {", ".join(temp_columns)} FROM {table};"  
    df_collection[table] = sql(query, engine)
    df_collection[table] = df_collection[table][df_collection[table].apply(
        lambda row: row.astype(str).str.contains(pattern, case=False, na=False).any(), axis=1
    )]
    

In [44]:
for table, df in df_collection.items():
    print(table)
    display(df)

accounts_profile


,id,language_id,section_id,user_id,position_eng,position_fre,position_from_ad
1,2,2.0,1.0,104,"Section Head, Aquaculture and Coastal Ecosystems","Chef de section, Aquaculture et écosystèmes côtiers","Section Head, Aquaculture and Coastal Ecosystems Section"
51,54,NaN,NaN,31,"Biologist, Aquaculture",Research Scientist / Chercheur,Research Scientist / Chercheur
67,70,NaN,NaN,64,"Head, Aquaculture & Coastal Ecosystems Section",None,None
76,79,NaN,NaN,89,Fisheries Biologist,Biologiste des pêches,None
78,81,NaN,NaN,92,"Senior Biologist, Aquaculture and Coastal Ecosystems","Division Manager, Aquatic Health / Gestionnaire de division, Santé aquatique","Division Manager, Aquatic Health / Gestionnaire de division, Santé aquatique"
...,...,...,...,...,...,...,...
3316,3326,NaN,NaN,3595,FPP Biologist biologiste - Aquatic Ecosystems,FPP Biologist biologiste - Aquatic Ecosystems,FPP Biologist biologiste - Aquatic Ecosystems
3337,3347,NaN,NaN,3616,Aquaculture Management Coordinator | Coordonnatrice de la gestion de l'aquaculture,Aquaculture Management Coordinator | Coordonnatrice de la gestion de l'aquaculture,Aquaculture Management Coordinator | Coordonnatrice de la gestion de l'aquaculture
3350,3360,NaN,NaN,3629,"Regional Director, Aquatic Ecosystems Directeur régional, Écosystèmes aquatiques","Regional Director, Aquatic Ecosystems Directeur régional, Écosystèmes aquatiques","Regional Director, Aquatic Ecosystems Directeur régional, Écosystèmes aquatiques"
3384,3394,NaN,NaN,3663,"Regional Senior Fisheries Management Officer,Resource Management","Regional Senior Fisheries Management Officer,Resource Management","Regional Senior Fisheries Management Officer,Resource Management"


bio_diversity_anidetsubjcode


,id,ANI_DET_ID,name
26,29,7,Aquaculture


csas2_csasrequest


...

,id,name
15,18,FAM - Aquaculture
19,48,Fisheries Management


spot_projectpurpose


,id,name
33,34,Support Food Social and Ceremonial Fisheries
44,47,Support Sustainable Aquaculture


tracking_email


,id,msg_id,sent_by_id,subject
12849,12850,010d01886d37237c-d65a6259-1c49-4e1d-ba6e-44cf2c0c39f8-000000,50.0,new ticket / nouveau billet - Whalebrary Marine Mammal Tools (Medium)
14654,14655,010d01894f26511c-b17292c0-2ce2-4706-bc4f-947e2ca4d24d-000000,452.0,new ticket / nouveau billet - Whalebrary Marine Mammal Tools (Urgent)
26936,26937,010d018f2f2d7d91-ad31b803-e854-4bd1-af0e-ac6dfea21360-000000,591.0,new ticket / nouveau billet - Whalebrary Marine Mammal Tools (Medium) ** ACCESS REQUEST **
27795,27796,010d01900266c74a-7f865ca1-8cff-4719-9bc2-d2feee2d9819-000000,3459.0,new ticket / nouveau billet - Whalebrary Marine Mammal Tools (Medium) ** ACCESS REQUEST **
28873,28874,010d019147726a9f-2bd64e59-67e5-449e-82b8-9cc7ced2ffcd-000000,467.0,FOR YOUR INFORMATION (no action required): Meeting 793: Science Review of Two Proposed Trout Aquaculture Sites in Bay d’Espoir on the South Coast of Newfoundland) - POUR VOTRE INFORMATION (aucune ...
28874,28875,010d019147825af5-78101095-c7d9-4894-a645-bbabe00cf677-000000,2547.0,FOR YOUR INFORMATION (no action required): Process 753: Science Review of Two Proposed Trout Aquaculture Sites in Bay d’Espoir on the South Coast of Newfoundland) - POUR VOTRE INFORMATION (aucune ...
29007,29008,010d01917049025f-05fd0853-4f89-4894-b6c6-c880f5bad0c0-000000,467.0,FOR YOUR INFORMATION (no action required): Process 753: Science Review of Two Proposed Trout Aquaculture Sites in Bay d’Espoir on the South Coast of Newfoundland) - POUR VOTRE INFORMATION (aucune ...
29260,29261,010d0191e812820e-b672f8c9-71af-4984-9e5d-80990921901b-000000,2422.0,FOR YOUR INFORMATION (no action required): Process 434: Science Review of Narwhal (Monodon Monoceros) Surveys Conducted by WPS Golder Inc. in 2019-2023 for use in DFO Fisheries Stock Assessments) ...
29261,29262,010d0191e814fa35-bae6f66d-31ce-4709-b9df-be6732fea60a-000000,2422.0,FOR YOUR INFORMATION (no action required): Process 434: Science Review of Narwhal (Monodon Monoceros) Surveys Conducted by WPS Golder Inc. in 2019-2023 for use in DFO Fisheries Stock Assessments) ...
29292,29293,010d0192007a1469-5ad2f388-8635-410f-9aae-d317a37db088-000000,2422.0,FOR YOUR INFORMATION (no action required): Process 434: Science Review of Narwhal (Monodon Monoceros) Surveys Conducted by WSP Golder Inc. in 2019-2023 for use in DFO Fisheries Stock Assessments) ...


In [47]:
potential_candidate_tables = ['ppt_csrftheme', 'ppt_functionalgroup', 'ppt_tag', 'publications_humancomponent']

for table in potential_candidate_tables:
    display(sql(f'SELECT * FROM {table}', engine))

,id,name,nom,code
0,7,Aquaculture Science,Science de l'aquaculture,AQ
1,8,Biotech & Genomics,Biotech & Génomique,BG
2,9,Aquatic Ecosystems Sci.: AIS-SARA,Science des écosystèmes aquatiques: EAE-LEP,AEa
3,10,"Aquatic Ecosystems Sci.: Ecosystems, habitats, impacts","Science des écosystèmes aquatiques: écosystèmes, habitats, impacts",AEb
4,11,Fisheries Science,Sciences halieutiques,FS
5,12,Climate Change & Oceans,None,CC
6,13,Aquatic Animal Health,None,AAH


,id,name,nom,theme_id
0,1,Administration and Coordination,Administration et coordination,7.0
1,2,Lobster,None,1.0
2,3,"Crustaceans, Other",None,1.0
3,4,Snow Crab,None,1.0
4,5,Microplastics and Cumulative Effects,Microplastiques et effets cumulatifs,3.0
...,...,...,...,...
72,80,Pacific Salmon Population Monitoring,None,1.0
73,81,Pacific Salmon Modelling & Decision Support,None,1.0
74,82,Hatchery Science,None,6.0
75,83,Contaminants,None,3.0


,id,name,nom
0,1,whales,baleine
1,2,eDNA,ADNe
2,3,ASEC,CESA
3,4,herring,hareng
4,5,groundfish,poisson de fond
...,...,...,...
327,331,Marine spatial planning,Planification spatiale marine
328,332,Newfoundland-Labrador Shelves Bioregion,Biorégions de plates-formes de Terre-Neuve et du Labrador
329,333,passive acoustic monitoring,surveillance acoustique passive
330,334,coastal,côtière


,id,name
0,104,AGRICULTURE
1,1,AQUACULTURE
2,102,AQUATIC INVASIVE SPECIES
3,10,CLIMATE CHANGE
4,103,COASTAL DEVELOPMENT
5,9,COASTAL INFRASTRUCTURE
6,37,DEBRIS
7,2,FISHERIES
8,106,HUMAN POPULATION DENSITY
9,11,HUMAN USE DATA


In [49]:
# no matches :(